In [9]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('../datasets/checking-logs.sqlite')
str10 =pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;',connection)

# print(str10)

query_min = """ SELECT t.uid, CAST (MIN(julianday(t.first_commit_ts)-
                    julianday(dl.deadlines,'unixepoch'))*24 as INTEGER) as 'MIN(diff)'
                    FROM test as t
                    LEFT JOIN deadlines as dl
                    ON t.labname = dl.labs
                    WHERE t.labname !='project1';"""

query_max = """ SELECT t.uid, CAST (MAX(julianday(t.first_commit_ts)-
                    julianday(dl.deadlines,'unixepoch'))*24 as INTEGER) as 'MAX(diff)'
                    FROM test as t
                    LEFT JOIN deadlines as dl
                    ON t.labname = dl.labs
                    WHERE t.labname !='project1';"""

query_avg = """ SELECT AVG(CAST((julianday(t.first_commit_ts)-
                    julianday(dl.deadlines,'unixepoch'))*24 as INTEGER)) AS 'AVG(diff)'
                    FROM test as t
                    LEFT JOIN deadlines as dl
                    ON t.labname = dl.labs
                    WHERE t.labname !='project1';"""

query_diff = """SELECT t.uid, t.avg_diff, p.pageviews 
                FROM (SELECT uid, AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) as avg_diff
                    FROM test
                    LEFT JOIN deadlines
                    ON test.labname = deadlines.labs
                    WHERE test.labname != 'project1'
                    GROUP BY uid) AS t
                LEFT JOIN 
                    (SELECT uid, COUNT(datetime) as pageviews
                    FROM pageviews  
                    WHERE uid LIKE 'user_%'
                    GROUP BY uid) AS p
                ON t.uid = p.uid"""


df_min= pd.io.sql.read_sql(query_min, connection)
df_min.to_sql('df_min',connection,if_exists='replace')

df_max= pd.io.sql.read_sql(query_max, connection)
df_max.to_sql('df_max',connection,if_exists='replace')

df_avg= pd.io.sql.read_sql(query_avg, connection)
df_avg.to_sql('df_avg',connection,if_exists='replace')

views_diff= pd.read_sql(query_diff,connection)

print(df_min)
print(df_max)
print(df_avg)
print(views_diff[['avg_diff','pageviews']].corr())

connection.close()

       uid  MIN(diff)
0  user_30       -202
       uid  MAX(diff)
0  user_25         -2
   AVG(diff)
0    -89.125
           avg_diff  pageviews
avg_diff   1.000000  -0.279736
pageviews -0.279736   1.000000
